In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
df_train = pd.read_csv('TrainingHandledData.csv')
df_test = pd.read_csv('TestingHandledData.csv')

In [3]:
df_test.shape

(1459, 75)

In [4]:
df_train.shape

(1451, 76)

In [5]:
df_final = pd.concat([df_train,df_test],axis=0)

In [6]:
df_final.shape

(2910, 76)

In [7]:
df_final.columns

Index(['Unnamed: 0', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea',
       'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF',

In [8]:
df_final.drop(['Unnamed: 0'],axis=1,inplace=True)

In [9]:
df_final = pd.get_dummies(df_final,drop_first=True)

In [10]:
df_final.shape

(2910, 236)

In [33]:
df_final.shape

(2910, 236)

In [34]:
Train = df_final.iloc[:1451,:]
Test = df_final.iloc[1451:,:]

In [35]:
Test.drop(['SalePrice'],axis=1,inplace=True)

C:\Users\amanm\AppData\Local\Temp\ipykernel_12252\1812229720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test.drop(['SalePrice'],axis=1,inplace=True)


In [36]:
X_train = Train.drop(['SalePrice'],axis = 1)
y_train = Train['SalePrice']

In [37]:
import xgboost
model = xgboost.XGBRegressor()
model.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [38]:
from sklearn.model_selection import RandomizedSearchCV

In [40]:
booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]
n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

In [41]:
random_cv = RandomizedSearchCV(estimator=model,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [42]:
random_cv.fit(X_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          num_parallel_tree=None,
                                          random_state=None, ...),
                   n_iter=50, n_jobs=4,
                   param_distributions={'base_score': [0.25, 0.5, 0.75, 1],
                                        'booster': ['gbtree', 'gblinear'],
                                        'learning_rate': [0.05, 0.1, 0.15, 0.2],
                                        'max_depth': [2, 3, 5, 10, 15],
                                        'min_child_weight': [1, 2, 3, 4],
                                        'n_estimators': [100, 500, 900, 1100,
                                                         1500]},
                   random_state=42, return_train_score=True,
                   scoring='neg_mean_absolute_error', verbose=5)

In [43]:
pred_train = model.predict(X_train)

In [44]:
model.score(X_train,y_train)

0.9995711560167333

In [45]:
import pickle
filename = 'finalmodel.pkl'

pickle.dump(model,open(filename,'wb'))

In [46]:
pred_test = pd.DataFrame(model.predict(Test))


In [47]:
sub_df = pd.read_csv('sample_submission.csv')

In [48]:
datasets=pd.concat([sub_df['Id'],pred_test],axis=1)
datasets.columns=['Id','SalePrice']
datasets

,Id,SalePrice
0,1461,130834.484375
1,1462,154081.703125
2,1463,178646.984375
3,1464,192936.062500
4,1465,210698.812500
...,...,...
1454,2915,81303.875000
1455,2916,71346.796875
1456,2917,163515.234375
1457,2918,118551.023438


In [49]:
datasets.isnull().sum()

Id           0
SalePrice    0
dtype: int64

In [50]:

datasets.to_csv('sample_submission.csv',index=False)